# Interactive Zero‑Knowledge Proofs for Graph Isomorphism

## 1. Problem Statement
We consider two public graphs, $G_1 = (V, E_1)$ and $G_2 = (V, E_2)$, where the vertex set is defined as $V=\{0, 1, \ldots, n-1\}$.

A **Prover** claims to know a specific permutation $\phi: V \to V$ (an isomorphism) such that:
$$G_2 = \phi(G_1)$$

The **Verifier** does not know $\phi$. The goal is for the Prover to convince the Verifier that such a $\phi$ exists without revealing any information about $\phi$ itself.

## 2. Basic Protocol
We utilize a standard interactive Zero-Knowledge protocol for Graph Isomorphism. The steps for a single round are as follows:

1. **Commitment:** The Prover chooses a random permutation $\sigma: V \to V$ and computes a commitment graph $H = \sigma(G_1)$. The Prover sends $H$ to the Verifier.
2. **Challenge:** The Verifier picks a random bit $c \in \{0, 1\}$ and sends it to the Prover.
3. **Response:**
   - If $c=0$, the Prover reveals $\sigma$.
   - If $c=1$, the Prover reveals $\rho = \sigma \circ \phi^{-1}$.
4. **Verification:**
   - If $c=0$, the Verifier checks if $H = \sigma(G_1)$.
   - If $c=1$, the Verifier checks if $H = \rho(G_2)$.

*Note: This basic version assumes an "Honest Verifier" who generates $c$ randomly. Malicious verifiers require more complex protocol modifications.*

## 3. Theoretical Background

### Zero-Knowledge Proof (ZKP)
A **Zero-Knowledge Proof** is a cryptographic method by which one party (the prover) can prove to another party (the verifier) that a given statement is true, without conveying any additional information apart from the fact that the statement is indeed true.

### Interactive Proof System
An **Interactive Proof System** models computation as an exchange of messages between two parties to determine the validity of a statement.
* **The Prover:** Possesses unlimited computational resources (theoretically) but is untrusted.
* **The Verifier:** Possesses bounded computational power (usually polynomial time) and is assumed to be honest.

The interaction continues until the Verifier is convinced. All interactive proof systems must satisfy two properties:

* **Completeness:** If the statement is true, an honest prover following the protocol will always convince the honest verifier.
* **Soundness:** If the statement is false, no cheating prover can convince the honest verifier that it is true, except with a negligibly small probability.

In [13]:
import numpy as np
import networkx as nx
import random
from typing import Union, List, Tuple

## Code #1: Mathematical Utilities and Verification
We define helper functions to handle graph permutations using adjacency matrices.

### Mathematical Context

**1. Adjacency Matrices**
An adjacency matrix $A$ for an undirected simple graph $G=(V,E)$ is a symmetric, square binary matrix where $A_{ij}=1$ if and only if there is an edge $\{i,j\}$. Key properties include:
* **Degree:** The degree of vertex $v_i$ is $d_i = \sum_{j=1}^n A_{ij}$.
* **Walks:** The $(i,j)$-th entry of $A^k$ represents the number of distinct walks of length $k$ between $v_i$ and $v_j$.
* **The Graph Laplacian ($L$):** Defined as $L = D - A$, or explicitly:
    $$
    L_{i,j} := 
    \begin{cases} 
        \deg(v_i) & \text{if } i = j \\
        -1 & \text{if } i \neq j \text{ and } v_i \text{ is adjacent to } v_j \\
        0 & \text{otherwise},
    \end{cases}
    $$

**2. Permutation Matrices**
A permutation matrix $P$ is derived by permuting the rows/columns of the identity matrix $I_n$. It satisfies:
* **Orthogonality:** $P^T = P^{-1}$.
* **Norm Preservation:** For any vector $x \in \{0,1\}^n$, $\|Px\|_2 = \|x\|_2$.
* **Graph Permutation:** If graph $H$ is a permutation of $G$ by $\sigma$, their adjacency matrices satisfy $M = P \cdot A \cdot P^T$ for some unique permutation matrix $P$. In our NumPy implementation, we achieve this efficiently via `G[sigma][:, sigma]`.

In [14]:
# --- Mathematical Utilities and Verification ---

def random_permutation(n: int) -> np.ndarray:
    """Generates a random permutation of size n."""
    sigma = np.arange(n)
    np.random.shuffle(sigma)
    return sigma

def apply_permutation(G_matrix: np.ndarray, sigma: np.ndarray) -> np.ndarray:
    """
    Apply the permutation sigma to the rows and columns of G.
    Mathematically: H = P * G * P^T
    """
    return G_matrix[sigma][:, sigma]

def invert_permutation(sigma: np.ndarray) -> np.ndarray:
    """Calculate the inverse permutation sigma^{-1}."""
    return np.argsort(sigma)

## Code #2: The Cheating Prover
This class simulates a malicious prover who does **not** know the isomorphism $\phi$ but attempts to fool the verifier.

**Strategy:**
Since the cheater lacks $\phi$, they cannot generate a commitment $H$ that maps to *both* $G_1$ and $G_2$ simultaneously. They must guess the verifier's challenge ($c=0$ or $c=1$) in advance.
* If they guess correctly, they can fake a valid response.
* If they guess incorrectly, they fail.
This serves as a control group to demonstrate the **Soundness** of the protocol.

In [15]:
class Cheater:
    def __init__(self, G1: np.ndarray, G2: np.ndarray):
        self.G1 = G1
        self.G2 = G2
        self.n = len(G1)
        self.sigma = None
        self.predicted_challenge = None

    def commit(self) -> np.ndarray:
        self.predicted_challenge = random.randint(0, 1)
        self.sigma = random_permutation(self.n)
        
        if self.predicted_challenge == 0:
            return apply_permutation(self.G1, self.sigma)
        else:
            return apply_permutation(self.G2, self.sigma)

    def respond(self, challenge: int) -> np.ndarray:
        if challenge != self.predicted_challenge:
            return self.sigma 
        return self.sigma

## Code #3: The Honest Prover
This class implements the correct Zero-Knowledge protocol logic.

### Protocol Logic
1.  **Initialization:** The prover knows $G_1, G_2$, and the secret isomorphism $\phi$ such that $G_2 = \phi(G_1)$.
2.  **Commitment:**
    - Generate a random permutation $\sigma$.
    - Compute the commitment graph $H = \sigma(G_1)$.
    - Send $H$ to the verifier.
3.  **Response:**
    - **Case $c=0$ (Show $H \cong G_1$):**
        The prover returns $\rho = \sigma$.
    - **Case $c=1$ (Show $H \cong G_2$):**
        We know $H = \sigma(G_1)$ and $G_1 = \phi^{-1}(G_2)$.
        Substituting $G_1$, we get $H = \sigma(\phi^{-1}(G_2))$.
        The prover returns the composed permutation $\rho = \sigma \circ \phi^{-1}$.

*Implementation Note:* In the code, `phi_inv` is computed via `np.argsort(phi)`.

In [16]:
class HonestProver:
    def __init__(self, G1: np.ndarray, G2: np.ndarray, phi: np.ndarray):
        self.G1 = G1
        self.G2 = G2
        self.phi = phi # The secret (isomorphism such that G2 = phi(G1))
        self.n = len(G1)
        self.sigma = None # Temporary commitment

    def commit(self) -> np.ndarray:
        """Step 1 : The Prover generates an isomorphic graph H."""
        self.sigma = random_permutation(self.n)
        H = apply_permutation(self.G1, self.sigma)
        return H

    def respond(self, challenge: int) -> np.ndarray:
        """Step 2 : The Prover responds to the challenge c."""
        if challenge == 0:
            return self.sigma
        else:
            phi_inv = invert_permutation(self.phi)
            return phi_inv[self.sigma]

## Code #4: The Verifier
The Verifier validates the Prover's response. While solving Graph Isomorphism is hard, verifying that two graphs are identical given a specific mapping is computationally easy ($O(n^2)$).

### Verification Logic
Given commitment $H$, challenge $c$, and response $\rho$:
1.  **If $c=0$:** Verify $\rho(G_1) \stackrel{?}{=} H$.
2.  **If $c=1$:** Verify $\rho(G_2) \stackrel{?}{=} H$.

### Soundness Analysis
A cheater has a $1/2$ probability of successfully guessing the challenge bit $c$ in any single round. By repeating the protocol for $k$ rounds, the probability of a cheater succeeding every time is $1/2^k$.
* For $k=20$, the probability of deception is $2^{-20} \approx 10^{-6}$.

In [17]:
class StrictVerifier:
    def __init__(self, G1: np.ndarray, G2: np.ndarray):
        self.G1 = G1
        self.G2 = G2

    def check(self, H: np.ndarray, rho: np.ndarray, challenge: int) -> bool:
        """Verify if rho transforms the target graph (G1 or G2) into H."""
        target_graph = self.G1 if challenge == 0 else self.G2
        
        # Calculate H_prime = rho(target_graph)
        # We removed the try/except block because performance validation was removed
        H_prime = apply_permutation(target_graph, rho)
            
        return np.array_equal(H_prime, H)

    def run_protocol(self, prover, rounds: int = 20) -> bool:
        print(f"--- Start of the protocol ({rounds} rounds) ---")
        for i in range(rounds):
            H = prover.commit()
            c = random.randint(0, 1)
            rho = prover.respond(c)
            
            if not self.check(H, rho, c):
                print(f"Failure at round {i+1} (Challenge c={c})")
                return False
                
        print("Success: The proof is accepted.")
        return True

## Simulation Execution
This utility generates the synthetic data required for the simulation:
1.  Creates a random Erdős‑Rényi graph $G_1$.
2.  Generates a random secret permutation $\phi$.
3.  Constructs the isomorphic graph $G_2 = \phi(G_1)$.

It returns the tuple $(G_1, G_2, \phi)$, acting as the ground truth.

We initialize the simulation with a graph size of $n = 100$. This size is sufficient to demonstrate the protocol without incurring high computational costs during testing.

**Expected Outcomes:**
1.  **Honest Prover:** Should satisfy the Verifier in all rounds (Result: `True`).
2.  **Cheating Prover:** Should fail eventually. With 30 rounds, the probability of the Cheater succeeding by pure luck is $2^{-30}$, which is effectively zero.

In [18]:
# --- Simulation ---

if __name__ == "__main__":
    n = 1000
    p = 0.3
    G1_nx = nx.erdos_renyi_graph(n, p)
    G1_matrix = nx.to_numpy_array(G1_nx)

    phi_secret = random_permutation(n)
    G2_matrix = apply_permutation(G1_matrix, phi_secret)

    verifier = StrictVerifier(G1_matrix, G2_matrix)

    print("Simulation with Cheater: ")
    cheater = Cheater(G1_matrix, G2_matrix)
    verifier.run_protocol(cheater, rounds=30)

    print("\nSimulation with Honest Prover: ")
    prover = HonestProver(G1_matrix, G2_matrix, phi_secret)
    verifier.run_protocol(prover, rounds=30)

Simulation with Cheater: 
--- Start of the protocol (30 rounds) ---
Failure at round 7 (Challenge c=1)

Simulation with Honest Prover: 
--- Start of the protocol (30 rounds) ---
Success: The proof is accepted.
